<a href="https://colab.research.google.com/github/Jorge2018/DataScience2022-2/blob/main/Script_regresionLinealBasica_Revisar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import re
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
df=pd.read_csv(r'https://raw.githubusercontent.com/Jorge2018/RepositorioArchivos/main/diabetes.csv')





In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


**Exploracion de datos**

In [25]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


**Busqueda de null**

In [4]:
# busqueda de null
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

**Separacion X y**

In [26]:
# separacion de df
X = df.drop(columns = 'BMI')
y = df['BMI']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 45)

In [27]:
# Instanciar los selectores de columnas categóricas y numéricas para seleccionar las columnas adecuadas
cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')

# Instanciar el escalador estándar y el codificador one hot
scaler = StandardScaler()
ohe = OneHotEncoder(handle_unknown='ignore')

# emparejamiento de cols con la tranformacion a utilizar. sobre valores numericos se utilizara StandardScaler, mientras sobre los categoricos OneHotEncoder
num_tuple = (scaler, num_selector)
cat_tuple = (ohe, cat_selector)

# instancia de col transformer, utilizando las tuplas creadas anteriormente
col_transformer = make_column_transformer(num_tuple, cat_tuple, remainder = 'passthrough')

# el transformador de cols solo sera utilizado en los datos de entrenamiento
col_transformer.fit(X_train)

# se procede con la transformacion de ambos df entrenamiento y prueba
X_train_processed = col_transformer.transform(X_train)
X_test_processed = col_transformer.transform(X_test)

X_train_processed

array([[ 1.21723833, -1.19866625,  0.25358478, ..., -0.06245673,
         0.49282404, -0.74616135],
       [ 1.80617514, -1.70811598,  1.84394702, ..., -0.58537677,
         1.17745871, -0.74616135],
       [-0.25510371, -0.05240434,  0.0547895 , ..., -0.07765789,
        -0.27738996, -0.74616135],
       ...,
       [-0.84404052, -1.07130381,  0.45238006, ..., -0.34215814,
        -0.36296929, -0.74616135],
       [ 0.0393647 , -1.00762259, -3.42412791, ...,  0.40269889,
        -0.19181062, -0.74616135],
       [-1.13850893,  0.52072662, -0.4421987 , ...,  0.1716412 ,
        -1.04760396,  1.34019271]])

In [28]:


# convierte en df y muestra los datos recien creados
X_train_df = pd.DataFrame(X_train_processed)
X_train_df.head()



,0,1,2,3,4,5,6,7
0,1.217238,-1.198666,0.253585,0.645332,-0.694543,-0.062457,0.492824,-0.746161
1,1.806175,-1.708116,1.843947,0.152265,-0.274900,-0.585377,1.177459,-0.746161
2,-0.255104,-0.052404,0.054789,0.583699,0.461616,-0.077658,-0.277390,-0.746161
3,-1.138509,-1.548913,-3.424128,-1.265305,-0.694543,-0.412083,-0.705287,-0.746161
4,1.217238,-0.052404,-3.424128,-1.265305,-0.694543,-0.895481,0.407245,1.340193


In [29]:
# Make a linear regression instance
reg = LinearRegression()
reg.fit(X_train,y_train)
print('El bias calculado es', reg.intercept_)
print('La pendiente calculada es: ', reg.coef_)

El bias calculado es 20.16808971892589
La pendiente calculada es:  [-2.35898900e-02  1.52918396e-02  8.62636626e-02  1.68092671e-01
 -1.12266458e-03  2.91901152e-01 -2.36714427e-02  3.74789054e+00]


**Codigo para obtener el R cuadrado (R^2), despues de realizado el ajuste sobre los datos de prueba**

In [30]:
test_score = reg.score(X_test, y_test)
print(test_score)

0.18174367236639177


**Codigo para obtener el R cuadrado (R^2), despues de realizado el ajuste sobre los datos de entrenamiento**

In [12]:
train_score = reg.score(X_train, y_train)
print(train_score)

0.2540582432095214


**Codigo para obtener las predicciones de los datos de Entrenamiento y prueba**

In [18]:
# Obtengan las predicciones del conjunto de entrenamiento
train_preds = reg.predict(X_train)
# Obtengan las predicciones del conjunto de prueba
test_preds = reg.predict(X_test)


[31.97269425 31.52866084 30.88365047 29.37852897 31.68920083 29.15098499
 35.96694449 37.76815763 26.89431048 34.18083547 35.02182466 30.79550587
 28.48348828 30.55279586 31.28695456 34.73026717 37.99260522 38.6362543
 31.96226784 27.64425635 31.95784808 26.98518635 37.78342083 28.93790416
 33.5486834  31.55763538 28.66031132 28.38190489 38.09287601 31.29970051
 28.43125749 30.63515572 26.21330042 22.67602336 27.32087612 38.47498123
 34.25724991 31.91935137 33.76779907 29.45955461 31.26254631 38.30936699
 30.62263762 30.43732983 29.65767798 26.91580102 36.25956989 27.97449828
 27.74304078 31.68820229 31.62977376 26.95443974 31.72609352 28.43830862
 35.28366366 28.05680246 26.36418498 33.34148426 34.82669479 28.92047184
 31.302472   28.83450394 29.32901429 30.98405998 28.99312155 30.26315443
 26.15770074 28.40774973 32.40575212 38.99910143 26.76063661 29.61427031
 27.22479864 30.90999447 31.81217357 33.09614033 32.28957113 28.48844728
 22.5196509  26.79423531 35.98581457 30.99396917 33.

**Error absoluto medio (EAM)**

In [22]:
mae_train = mean_absolute_error(y_train, train_preds)
mae_test = mean_absolute_error(y_test, test_preds)
print(mae_train)
print(mae_test)

4.883108198487533
5.36110039669576


In [ ]:
mse_train = mean_squared_error(y_train, train_preds)
mse_test = mean_squared_error(y_test, test_preds)

**Error cuadratico**